In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from mpl_toolkits.mplot3d import axes3d

import matplotlib
import time
from matplotlib.widgets import Slider, Button

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

In [2]:
print(np.linspace(0,20,21))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]


In [124]:
matplotlib.use('tkagg')
class mode_plot():
    def __init__(self,DOFs_sensors,phi):
        #moves the first tower to the beginning of the list 
        DOFs_sensors=DOFs_sensors[[16,17,0,1,2,3,4,5,6,7,8,
                                9,10,11,12,13,14,15,18,19]]

        # [H10E,H10W,H1E,H1W,H2W,H3E,H3W,H4E,H4W,H5E,H5W,H6E,
        #   H6W,H7E,H7W,H8E,H9E,H9W,H11E,H11W]

        self.DOFs_sensors=DOFs_sensors

        #interpolate for the two missing sensors in the bridge deck. 
        H2E_int= (DOFs_sensors[2]+DOFs_sensors[5])/2
        H8W_int = (DOFs_sensors[14]+DOFs_sensors[17])/2

        #Defining the structures base points 
        self.base_H10E=np.array([655  ,  7,   -60]) 
        self.base_H10W=np.array([655  , -7.3, -60]) 
        self.base_H11E=np.array([-655 ,  7,   -60]) 
        self.base_H11W=np.array([-655,  -7.3, -60]) 

        #dummy pinots between base an top of tower to connect bridge deck to
        dummy_H10E =self.dummy_point(DOFs_sensors[0],self.base_H10E,-10.6)  
        dummy_H10W=self.dummy_point(DOFs_sensors[1],self.base_H10W,-10.6)
        dummy_H11E=self.dummy_point(DOFs_sensors[-2],self.base_H11E,-10.6)
        dummy_H11W=self.dummy_point(DOFs_sensors[-1],self.base_H11W,-10.6) 

        #adding interpolated points to DOFs 
        self.DOFs=np.insert(DOFs_sensors,4,H2E_int,axis=0)
        self.DOFs=np.insert(self.DOFs,17,H8W_int,axis=0)
  
        #adding base and dummy point for tower 1:
        self.DOFs=np.insert(self.DOFs,2,self.base_H10E,axis=0)
        self.DOFs=np.insert(self.DOFs,3,self.base_H10W,axis=0)
        self.DOFs=np.insert(self.DOFs,4,dummy_H10E,axis=0)
        self.DOFs=np.insert(self.DOFs,5,dummy_H10W,axis=0)
        
        #adding base and dummy point for tower 2        
        self.DOFs=np.insert(self.DOFs,24,dummy_H11E,axis=0)
        self.DOFs=np.insert(self.DOFs,25,dummy_H11W,axis=0)
        self.DOFs=np.insert(self.DOFs,26,self.base_H11E,axis=0)
        self.DOFs=np.insert(self.DOFs,27,self.base_H11W,axis=0)
        
        #setting up the plot
        self.x0=self.DOFs[:,0]
        self.y0=self.DOFs[:,1]
        self.z0=self.DOFs[:,2]
        self.phi=self.sort_phi(phi)
        self.fig = plt.figure(figsize=(10,10))
        
        # self.ax=plt.subplot2grid((1,1),(0,0),projection='3d',fig=self.fig)
        self.ax=self.fig.add_subplot(111,projection='3d')
        self.ax.set_ylim(-100,100)
        self.ax.set_xlim(-700,700)
        self.ax.set_zlim(-20,10)
        self.ax.set_box_aspect([100,35,3])
        self.ax.set_axis_off()
        plt.subplots_adjust(left=-0.4, bottom=-0.4,
        right=1.4, top=1.4, wspace=0, hspace=0)

        # plotting initial configuration 

        self.dots = self.ax.plot(self.x0,self.y0,
        self.z0,marker='.',
        markersize=12,linestyle='None')[0]

        self.dots.set_markevery([0,1,6,7,9,10,11,12,13,14,15,16,17,18,19,20
        ,22,23,28,29])

        self.dots.set_markerfacecolor('peru')
        self.dots.set_markeredgecolor('peru')

        X=np.array([self.x0[::2],self.x0[1::2]])
        Y=np.array([self.y0[::2],self.y0[1::2]])
        Z=np.array([self.z0[::2],self.z0[1::2]])

        self.lines=self.ax.plot_wireframe(X,Y,Z,
        rstride=1,cstride=1,color='black',linewidth=2.5)

        ints=np.array([H2E_int,H8W_int,dummy_H10E,dummy_H10W
        ,dummy_H11E,dummy_H11W])
        self.dots_int=self.ax.plot(ints[:,0],ints[:,1],ints[:,2],
        marker='.',markersize=12,linestyle='None',alpha=1,color='grey')[0]

        ##############################################
        #plotting undeformed configuration to stay in bg
        self.dots_perm = self.ax.plot(self.x0,self.y0,self.z0,
        marker='.',linestyle='None',color='grey',alpha=0.2)[0]

        b_ix=[26,27,2,3]
        self.dots_perm2 = self.ax.plot(self.x0[b_ix],self.y0[b_ix],
        self.z0[b_ix],marker='s',linestyle='None',
        color='black',alpha=0.5)[0]

        X=np.array([self.x0[::2],self.x0[1::2]])
        Y=np.array([self.y0[::2],self.y0[1::2]])
        Z=np.array([self.z0[::2],self.z0[1::2]])
        self.lines_perm=self.ax.plot_wireframe(X,Y,Z,
        rstride=1,cstride=1,color='grey',alpha=0.2)
  
        #setting up the sliders

        axfreq = self.fig.add_axes([0.05, 0.05, 0.03, 0.6])
        self.fac_slider = Slider(
        ax=axfreq,
        label='Amplification\n factor',
        valmin=0,
        valmax=110,
        valinit=10,
        orientation="vertical")

        self.axfreq = self.fig.add_axes([0.1, 0.05, 0.03, 0.6])
        self.t_slider = Slider(
        ax=self.axfreq,
        label='Phase',
        valmin=-1,
        valmax=1,
        valinit=1,
        orientation="vertical")

        self.fac_slider.on_changed(self.update_DOFs)
        self.t_slider.on_changed(self.update_DOFs)

    def update_DOFs(self,event):
        fac=self.fac_slider.val*self.t_slider.val
        DOFs_i=np.zeros_like(self.DOFs_sensors[:])
        DOFs_i[:,0]=self.DOFs_sensors[:,0]+self.phi[0::3]*fac
        DOFs_i[:,1]=self.DOFs_sensors[:,1]+self.phi[1::3]*fac
        DOFs_i[:,2]=self.DOFs_sensors[:,2]+self.phi[2::3]*fac


        #interpolate for the two missing sensors in the bridge deck. 
        H2E_int_i= (DOFs_i[2]+DOFs_i[5])/2
        H8W_int_i = (DOFs_i[14]+DOFs_i[17])/2

        #dummy pinots between base an top of tower to connect bridge deck to
        dummy_H10E =self.dummy_point(DOFs_i[0],self.base_H10E,-10.6)  
        dummy_H10W=self.dummy_point(DOFs_i[1],self.base_H10W,-10.6)
        dummy_H11E=self.dummy_point(DOFs_i[-2],self.base_H11E,-10.6)
        dummy_H11W=self.dummy_point(DOFs_i[-1],self.base_H11W,-10.6) 

        #adding interpolated points to DOFs 
        DOFs_i=np.insert(DOFs_i,4,H2E_int_i,axis=0)
        DOFs_i=np.insert(DOFs_i,17,H8W_int_i,axis=0)
  
        #adding base and dummy point for tower 1:
        DOFs_i=np.insert(DOFs_i,2,self.base_H10E,axis=0)
        DOFs_i=np.insert(DOFs_i,3,self.base_H10W,axis=0)
        DOFs_i=np.insert(DOFs_i,4,dummy_H10E,axis=0)
        DOFs_i=np.insert(DOFs_i,5,dummy_H10W,axis=0)
        
        #adding base and dummy point for tower 2        
        DOFs_i=np.insert(DOFs_i,24,dummy_H11E,axis=0)
        DOFs_i=np.insert(DOFs_i,25,dummy_H11W,axis=0)
        DOFs_i=np.insert(DOFs_i,26,self.base_H11E,axis=0)
        DOFs_i=np.insert(DOFs_i,27,self.base_H11W,axis=0)

        ints=np.array([H2E_int_i,H8W_int_i])

        # updating the sensor DOFs
        self.dots.set_data_3d(DOFs_i[:,0],DOFs_i[:,1],DOFs_i[:,2])

        ints=np.array([H2E_int_i,H8W_int_i,dummy_H10E,dummy_H10W
        ,dummy_H11E,dummy_H11W])
        self.dots_int.set_data_3d(ints[:,0],ints[:,1],ints[:,2])

        #updating wireframe
        X=np.array([DOFs_i[0::2,0],DOFs_i[1::2,0]])
        Y=np.array([DOFs_i[0::2,1],DOFs_i[1::2,1]])
        Z=np.array([DOFs_i[0::2,2],DOFs_i[1::2,2]])

        #drawing new wireframe
        self.lines.remove()
        self.lines=self.ax.plot_wireframe(X, Y, Z, cstride=1,
                          rstride=1, color="black",linewidth=2.5)

        self.fig.canvas.draw()
    
    def dummy_point(self,st,sb,z): 
        h_fac=(z-sb[2])/(st[2]-sb[2])  #from bot / tot heigth
        x=sb[0]+(st[0]-sb[0])*h_fac
        y=sb[1]+(st[1]-sb[1])*h_fac
        return np.array([x,y,z])
        
    def show(self):
        self.axfreq.legend([self.dots,self.dots_int,self.dots_perm2],
        ['Sensor placements','Interpolated points','Supports'],
        loc='center', bbox_to_anchor=(3.5, 0.05))
        mng = plt.get_current_fig_manager()
        mng.window.state('zoomed')
        plt.show()   
    
    def sort_phi(self,phi):
        return phi[[48,49,50,51,52,53,0,1,2,3,4,5,6,7,8,9,10,
                        11,12,13,14,15,16,17,18,19,20,21,22,
                        23,24,25,26,27,28,29,30,31,32,33,34,35,
                        36,37,38,39,40,41,42,43,44,45,46,47,54,
                        55,56,57,58,59]]
        
        

In [4]:
'''PHI:
[H1E_x, H1E_y,  H1E_z,  H1W_x,  H1W_y,  H1W_z,  H2W_x,  H2W_y,  H2W_z, 
H3E_x,  H3E_y,  H3E_z,  H3W_x,  H3W_y,  H3W_z,  H4E_x,  H4E_y,  H4E_z, 
H4W_x,  H4W_y,  H4W_z,  H5E_x,  H5E_y,  H5E_z,  H5W_x,  H5W_y,  H5W_z, 
H6E_x,  H6E_y,  H6E_z,  H6W_x,  H6W_y,  H6W_z,  H7E_x,  H7E_y,  H7E_z, 
H7W_x,  H7W_y,  H7W_z,  H8E_x,  H8E_y,  H8E_z,  H9E_x,  H9E_y,  H9E_z, 
H9W_x,  H9W_y,  H9W_z, H10E_x, H10E_y, H10E_z, H10W_x, H10W_y, H10W_z, 
H11E_x, H11E_y, H11E_z,H11W_x, H11W_y, H11W_z]

H1-H9:   bridge deck
H10-H11: Towers

slicing axamples: 
#slicing:

bridge_west_side_x=phi[:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_x=phi[:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

bridge_west_side_y=phi[1:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_y=phi[1:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

bridge_west_side_z=phi[2:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_z=phi[2:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

tower_10=phi[16*3:18*3]
tower_11=phi[18*3:20*3]
'''

def create_phi(x,y,z):
    '''Create a phi vector based on x, y, z - vectors'''
    phi=[]
    for i in range(len(x)): 
        phi.append(x[i])
        phi.append(y[i])
        phi.append(z[i])
    return np.array(phi)


t=np.linspace(0,20,20)
z=np.sin(t)

phi=create_phi(z*0,z*0,z)
print(len(phi))
# test=mode_plot(DOFs_sensors,phi)
# test.show()

60


In [5]:
H1E=np.array([ 480.0, 6.33,-8.38])  # 0
H1W=np.array([ 480.0,-6.64,-8.38])
H2W=np.array([ 360.0,-6.64,-6.64]) 
H3E=np.array([ 240.0, 6.33,-4.45])
H3W=np.array([ 240.0,-6.64,-4.45]) 
H4E=np.array([ 120.0, 6.33,-2.48]) 
H4W=np.array([ 120.0,-6.64,-2.48]) 
H5E=np.array([-7.0  , 6.33, -0.4]) 
H5W=np.array([-7.0  ,-6.64, -0.4]) 
H6E=np.array([-120.0, 6.33,-2.25]) 
H6W=np.array([-120.0,-6.64,-2.25]) 
H7E=np.array([-240.0, 6.33,-4.22]) 
H7W=np.array([-240.0,-6.64,-4.22]) 
H8E=np.array([-360.0, 6.33,-6.18]) 
H9E=np.array([-480.0, 6.33,-8.15]) 
H9W=np.array([-480.0,-6.64,-8.15]) # 15

H10E=np.array([655  ,  4.5, 120.5]) 
H10W=np.array([655  , -4.5, 120.5]) 
H11E=np.array([-655 ,  4.5, 120.5]) 
H11W=np.array([-655,  -4.5, 120.5]) 

DOFs_sensors=np.array([H1E,H1W,H2W,H3E,H3W,H4E,H4W,H5E,H5W,H6E,H6W,H7E,H7W,H8E,H9E,H9W,H10E,H10W,H11E,H11W])




In [6]:
# backup: 

    def update_DOFs(self,event):
 
        fac=self.fac_slider.val*self.t_slider.val
        DOFs_i=np.zeros_like(self.DOFs_sensors[:])
        DOFs_i[:,0]=self.DOFs_sensors[:,0]+self.phi[0::3]*fac
        DOFs_i[:,1]=self.DOFs_sensors[:,1]+self.phi[1::3]*fac
        DOFs_i[:,2]=self.DOFs_sensors[:,2]+self.phi[2::3]*fac

        H2E_int= (DOFs_i[0]+DOFs_i[3])/2
        H8W_int = (DOFs_i[12]+DOFs_i[15])/2

        ints=np.array([H2E_int,H8W_int])

        DOFs_i=np.insert(DOFs_i,2,H2E_int,axis=0)
        DOFs_i=np.insert(DOFs_i,15,H8W_int,axis=0)

        # updating the sensor DOFs
        self.dots.set_xdata(DOFs_i[:,0].ravel())
        self.dots.set_ydata(DOFs_i[:,1].ravel())
        self.dots.set_3d_properties(DOFs_i[:,2].ravel())

        #Updating marking of interpolated DOFs
        self.dots_int.set_xdata(ints[:,0].ravel())
        self.dots_int.set_ydata(ints[:,1].ravel())
        self.dots_int.set_3d_properties(ints[:,2].ravel())

        #updating wireframe
        X=np.array([DOFs_i[0::2,0],DOFs_i[1::2,0]])
        Y=np.array([DOFs_i[0::2,1],DOFs_i[1::2,1]])
        Z=np.array([DOFs_i[0::2,2],DOFs_i[1::2,2]])
        #drawing new wireframe
        self.lines.remove()
        self.lines=self.ax.plot_wireframe(X, Y, Z, cstride=1,
                          rstride=1, color="black")

        self.fig.canvas.draw()

IndentationError: unexpected indent (559081430.py, line 3)

In [8]:
phi=np.array(['H1E_x', 'H1E_y', 'H1E_z', 'H1W_x', 'H1W_y', 'H1W_z', 'H2W_x', 'H2W_y', 'H2W_z', 'H3E_x', 'H3E_y', 'H3E_z', 'H3W_x', 'H3W_y', 'H3W_z', 'H4E_x', 'H4E_y', 'H4E_z', 'H4W_x', 'H4W_y', 'H4W_z', 'H5E_x', 'H5E_y', 'H5E_z', 'H5W_x', 'H5W_y', 'H5W_z', 'H6E_x', 'H6E_y', 'H6E_z', 'H6W_x', 'H6W_y', 'H6W_z', 'H7E_x', 'H7E_y', 'H7E_z', 'H7W_x', 'H7W_y', 'H7W_z', 'H8E_x', 'H8E_y', 'H8E_z', 'H9E_x', 'H9E_y', 'H9E_z', 'H9W_x', 'H9W_y', 'H9W_z', 'H10E_x', 'H10E_y', 'H10E_z', 'H10W_x', 'H10W_y', 'H10W_z', 'H11E_x', 'H11E_y', 'H11E_z', 'H11W_x', 'H11W_y', 'H11W_z'])

#slicing:
bridge_west_side_x=phi[:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_x=phi[:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

bridge_west_side_y=phi[1:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_y=phi[1:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

bridge_west_side_z=phi[2:16*3:3][[1,2,4,6,8,10,12,15]] #len=8
bridge_east_side_z=phi[2:16*3:3][[0,3,5,7,9,11,13,14]] #len=8

tower_10=phi[16*3:18*3]
tower_11=phi[18*3:20*3]

print(bridge_west_side_x)
print(bridge_east_side_x)

print(bridge_west_side_y)
print(bridge_east_side_y)

print(bridge_west_side_z)
print(bridge_east_side_z)

print(tower_10)
print(tower_11)

# creating phi: 
phi=np.zeros(60)
y_shape0=np.sin(np.linspace(0,2*np.pi,11))
y_shape=y_shape0[1:-1]

phi[1:16*3:3][[1,2,4,6,8,10,12,15]]=y_shape[[0,1,2,3,4,5,6,8]] 
phi[1:16*3:3][[0,3,5,7,9,11,13,14]]=y_shape[[0,2,3,4,5,6,7,8]]
# print(phi)





['H1W_x' 'H2W_x' 'H3W_x' 'H4W_x' 'H5W_x' 'H6W_x' 'H7W_x' 'H9W_x']
['H1E_x' 'H3E_x' 'H4E_x' 'H5E_x' 'H6E_x' 'H7E_x' 'H8E_x' 'H9E_x']
['H1W_y' 'H2W_y' 'H3W_y' 'H4W_y' 'H5W_y' 'H6W_y' 'H7W_y' 'H9W_y']
['H1E_y' 'H3E_y' 'H4E_y' 'H5E_y' 'H6E_y' 'H7E_y' 'H8E_y' 'H9E_y']
['H1W_z' 'H2W_z' 'H3W_z' 'H4W_z' 'H5W_z' 'H6W_z' 'H7W_z' 'H9W_z']
['H1E_z' 'H3E_z' 'H4E_z' 'H5E_z' 'H6E_z' 'H7E_z' 'H8E_z' 'H9E_z']
['H10E_x' 'H10E_y' 'H10E_z' 'H10W_x' 'H10W_y' 'H10W_z']
['H11E_x' 'H11E_y' 'H11E_z' 'H11W_x' 'H11W_y' 'H11W_z']


In [9]:
l=[48,49,50,51,52,53,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,
23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,54,
55,56,57,58,59]

for p in l:
    print(p,'   ',p//3)

48     16
49     16
50     16
51     17
52     17
53     17
0     0
1     0
2     0
3     1
4     1
5     1
6     2
7     2
8     2
9     3
10     3
11     3
12     4
13     4
14     4
15     5
16     5
17     5
18     6
19     6
20     6
21     7
22     7
23     7
24     8
25     8
26     8
27     9
28     9
29     9
30     10
31     10
32     10
33     11
34     11
35     11
36     12
37     12
38     12
39     13
40     13
41     13
42     14
43     14
44     14
45     15
46     15
47     15
54     18
55     18
56     18
57     19
58     19
59     19


In [123]:
test2=mode_plot(DOFs_sensors,phi)
test2.show()